<a href="https://colab.research.google.com/github/TongleiChen/colab_notebook/blob/main/a4_languagemodel_lstm_template_0403_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A4 - Language Model LSTM 

Author: Austin Blodgett

Adaptation to colab: Nitin Venkateswaran

### Follow the steps to use this notebook for your A4.

**NOTE**: It is best to use your Georgetown Google accounts.
##### 1. Save a copy of this notebook starter template in your Google Drive (File -> Save a copy in drive)
##### 2. Upload a copy of all 3 txt files from **lm-data** directory (available in a4.zip) to your Google Drive in the folder location **A4/lm-data/**; you will need to create the folder 'A4' at the root location in your Drive, followed by the subfolder 'lm-data'
##### 3. You are all set!


###Import libraries and mount Google Drive





In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.11.0

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers
import os, random
from collections import Counter

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed

import numpy as np
import tensorflow as tf
from keras import backend as K
from keras import Model
from keras.activations import softmax
from keras.initializers import Constant

from transformers import BertTokenizer, TFBertLMHeadModel, BertConfig, TFBertModel
import tensorflow as tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_file = '/content/drive/My Drive/A4/lm-data/little-prince-train.txt'
dev_file = '/content/drive/My Drive/A4/lm-data/little-prince-dev.txt'
test_file = '/content/drive/My Drive/A4/lm-data/little-prince-test.txt'
UNK = '[UNK]'
PAD = '[PAD]'
START = '<s>'
END = '</s>'
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

###Change these arguments as needed for your experiments

In [ ]:
epochs = 7 # number of epochs
learning_rate = 0.01 # learning rate
early_stopping = -1 # early stopping criteria
embedding_size = 100 # embedding dimension size
hidden_size = 100 # hidden layer size
batch_size = 50 # batch size
use_bert = False # to use the BERT embeddings

### Implement this function if you want to transform the input text, e.g. normalizing case


In [ ]:
# TODO
def transform_text_sequence(seq):
    '''
    Implement this function if you want to transform the input text,
    for example normalizing case.
    '''
    return seq

### Implement this function to generate the next-word labels for a sequence

In [ ]:
def shift_by_one(seq):
    '''
    input: ['<s>', 'The', 'dog', 'chased', 'the', 'cat', 'around', 'the', 'house', '</s>']
    output: ['The', 'dog', 'chased', 'the', 'cat', 'around', 'the', 'house', '</s>', '[PAD]']
    '''
    output = []
    for i in range(1,len(seq)):
      output.append(seq[i])
      
    output.append(PAD)
    return output



### Download the GloVe embeddings

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip -o glove.6B.zip

--2023-04-02 17:03:21--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-02 17:03:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2023-04-02 17:06:00 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

### Implement this function to load the pre-trained GloVE embeddings

In [ ]:
glove_file = 'glove.6B.100d.txt' # Change as necessary

def load_pretrained_embeddings(glove_file, vocab):
    embedding_matrix = np.zeros((len(vocab), embedding_size))
    embeddings_index = {}
    word_index = dict(zip(vocab, range(len(vocab))))
    with open(glove_file, encoding='utf8') as f:
        for line in f:
            # Each line will be a word and a list of floats, separated by spaces.
            # If the word is in your vocabulary, create a numpy array from the list of floats.
            # Assign the array to the correct row of embedding_matrix.
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    num_tokens = len(vocab)


    # Prepare embedding matrix
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector


            
    embedding_matrix[vocab[UNK]] = np.random.randn(embedding_size)
    return embedding_matrix

###Helper Functions (no need to implement)



In [ ]:
def get_vocabulary_and_data_with_bert_tokenization(data_file):
    data = []
    with open(data_file, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            sent = [START]
            sent.extend(tokenizer.tokenize(line))
            sent.append(END)
            data.append(sent)
    vocab = {k:v for k,v in tokenizer.vocab.items()}
    vocab['<s>'] = 101 # alias for [CLS]
    vocab['</s>'] = 102 # alias for [SEP]
    return vocab, data


def get_vocabulary_and_data(data_file, max_vocab_size=None, use_bert=False):
    if use_bert:
        return get_vocabulary_and_data_with_bert_tokenization(data_file)
    vocab = Counter()
    data = []
    with open(data_file, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            sent = [START]
            tokens = transform_text_sequence(line.split())
            for tok in tokens:
                sent.append(tok)
                vocab[tok]+=1
            sent.append(END)
            data.append(sent)
            vocab[START]+=1
            vocab[END]+=1
    vocab = [w for w in sorted(vocab, key=lambda x:vocab[x], reverse=True)]
    if max_vocab_size:
        vocab = vocab[:max_vocab_size-2]
    vocab = [UNK, PAD] + vocab

    return {k:v for v,k in enumerate(vocab)}, data


def vectorize_sequence(seq, vocab):
    seq = [tok if tok in vocab else UNK for tok in seq]
    return [vocab[tok] for tok in seq]


def unvectorize_sequence(seq, vocab):
    translate = sorted(vocab.keys(),key=lambda k:vocab[k])
    return [translate[i] for i in seq]


def one_hot_encode_label(label, vocab):
    vec = [1.0 if l==label else 0.0 for l in vocab]
    return vec


def batch_generator_lm(data, vocab, batch_size=1):
    while True:
        batch_x = []
        batch_y = []
        for sent in data:
            batch_x.append(vectorize_sequence(sent, vocab))
            batch_y.append([one_hot_encode_label(token, vocab) for token in shift_by_one(sent)])
            if len(batch_x) >= batch_size:
                # Pad Sequences in batch to same length
                batch_x = pad_sequences(batch_x, vocab[PAD])
                batch_y = pad_sequences(batch_y, one_hot_encode_label(PAD, vocab))
                batch_x, batch_y = np.array(batch_x), np.array(batch_y)
                yield batch_x, batch_y.astype('float32')
                batch_x = []
                batch_y = []


def describe_data(data, generator):
    batch_x, batch_y = [], []
    for bx, by in generator:
        batch_x = bx
        batch_y = by
        break
    print('Data example:',data[49])
    print('Data size',len(data))
    print('Batch input shape:', batch_x.shape)
    print('Batch output shape:', batch_y.shape)


def pad_sequences(batch_x, pad_value):
    ''' This function should take a batch of sequences of different lengths
        and pad them with the pad_value token so that they are all the same length.

        Assume that batch_x is a list of lists.
    '''
    pad_length = len(max(batch_x, key=lambda x: len(x)))
    for i, x in enumerate(batch_x):
        if len(x) < pad_length:
            batch_x[i] = x + ([pad_value] * (pad_length - len(x)))

    return batch_x


def generate_text(language_model, vocab):
    prediction = [START]
    while not (prediction[-1] == END or len(prediction)>=50):
        next_token_one_hot = language_model.predict(np.array([[vocab[p] for p in prediction]]), batch_size=1)[0][-1]
        threshold = random.random()
        sum = 0
        next_token = 0
        for i,p in enumerate(next_token_one_hot):
            sum += p
            if sum>threshold:
                next_token = i
                break
        for w, i in vocab.items():
            if i==next_token:
                prediction.append(w)
                break
    return prediction


def perplexity(y_true, y_pred):
    # https://stackoverflow.com/questions/41881308/how-to-calculate-perplexity-of-rnn-in-tensorflow
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perp = K.exp(cross_entropy)
    return perp


class BERT_Wrapper(Model):

  def __init__(self):
    super(BERT_Wrapper, self).__init__()
    self.encoder = TFBertModel.from_pretrained("bert-base-uncased", trainable=False)
    self.dense = Dense(hidden_size)

  def call(self, inputs, **kwargs):
      outputs = self.encoder(inputs)
      last_hidden_states = outputs[0] # The last hidden-state is the first element of the output tuple
      output = self.dense(last_hidden_states)
      return output

In [ ]:
# BERT_Wrapper()

###Check the GPU is available

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  device_name = '/cpu:0'
  print(
      '\n\n This notebook is not '
      'configured to use a GPU.  You can change this in Notebook Settings. Defaulting to:' + device_name)
else:
  print ('GPU Device found: ' + device_name)

GPU Device found: /device:GPU:0


###Main procedure call: Implement the keras model here


In [ ]:
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'next', ',', 'the', 'lamp', '##light', '##ers', 'of', 'china', 'and', 'siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 33)
Batch output shape: (50, 33, 30524)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1 / 3
100/100 [==============================] - 15s 96ms/step - loss: 7.2329 - accuracy: 0.0612 - perplexity: 684407.5625
Dev Loss: 7.232909679412842 Dev Acc: 0.06123698875308037 Dev Peprlexity: 684407.5625
Epoch 2 / 3
100/100 [==============================] - 11s 114ms/step - loss: 5.6400 - accuracy: 0.0637 - perplexity: 41809.3125
Dev Loss: 5.639987945556641 Dev Acc: 0.06368646770715714 Dev Peprlexity: 41809.3125
Epoch 3 / 3
100/100 [==============================] - 10s 103ms/step - loss: 5.4347 - accuracy: 0.0961 - perplexity: 52570.8516
Dev Loss: 5.4346923828125 Dev Acc: 0.09614206850528717 Dev Peprlexity: 52570.8515625
1/1 [==============================] - 0s 49ms/step
['<s>', 'bobbie', 'bmw', 'now', 'prince', '"', 'dude', 'prince', 'sunrise', '[PAD]', 'it', 'sad', 'high', 'bo', 'million', '"', 'they', 'i', '##nova', 'the', 'and', 'aziz', '[PAD]', 'know', '[PAD]', 'me', 'heads', 'the', 'introducing', 'bucharest', 'i', '[PAD]', 'was', '##baldi', 'with', 'prince', 'to', 'f

In [ ]:
epochs = 5 # number of epochs
learning_rate = 0.01 # learning rate

with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1 / 5
100/100 [==============================] - 14s 98ms/step - loss: 6.6980 - accuracy: 0.0612 - perplexity: 4561.8823
Dev Loss: 6.697998523712158 Dev Acc: 0.06123698875308037 Dev Peprlexity: 4561.88232421875
Epoch 2 / 5
100/100 [==============================] - 8s 84ms/step - loss: 5.7876 - accuracy: 0.0612 - perplexity: 10325.8076
Dev Loss: 5.78758430480957 Dev Acc: 0.06123698875308037 Dev Peprlexity: 10325.8076171875
Epoch 3 / 5
100/100 [==============================] - 13s 125ms/step - loss: 6.2371 - accuracy: 0.0612 - perplexity: 38288.0820
Dev Loss: 6.237147808074951 Dev Acc: 0.06123698875308037 Dev Peprlexity: 38288.08203125
Epoch 4 / 5
100/100 [==============================] - 10s 103ms/step - loss: 6.0472 - accuracy: 0.0612 - perplexity: 59609.7148
Dev Loss: 6.047185897827148 Dev Acc: 0.06123698875308037 Dev Peprlexity: 59609.71484375
Epoch 5 / 5
100/100 [==============================] - 11s 106ms/step - loss: 6.0735 - accuracy: 0.0612 - perplexity: 79158.9844
Dev 

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 15
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 15
100/100 [==============================] - 2s 11ms/step - loss: 4.8476 - accuracy: 0.0936 - perplexity: 116598.4844
Dev Loss: 4.8476176261901855 Dev Acc: 0.09361163526773453 Dev Peprlexity: 116598.484375
Epoch 2 / 15
100/100 [==============================] - 1s 12ms/step - loss: 4.3492 - accuracy: 0.1746 - perplexity: 16956.7598
Dev Loss: 4.349239349365234 Dev Acc: 0.17457304894924164 Dev Peprlexity: 16956.759765625
Epoch 3 / 15
100/100 [==============================] - 1s 7ms/step - loss: 4.0823 - accuracy: 0.1999 - perplexity: 3358.7937
Dev Loss: 4.08231782913208 Dev Acc: 0.19987349212169647 Dev Peprlexity: 3358.793701171875
Epoch 4 / 15

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 10
hidden_size = 5
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 10
100/100 [==============================] - 1s 7ms/step - loss: 4.8108 - accuracy: 0.0633 - perplexity: 17685.8105
Dev Loss: 4.810844898223877 Dev Acc: 0.06325110793113708 Dev Peprlexity: 17685.810546875
Epoch 2 / 10
100/100 [==============================] - 1s 8ms/step - loss: 4.5948 - accuracy: 0.1347 - perplexity: 8143.6191
Dev Loss: 4.594799041748047 Dev Acc: 0.13472485542297363 Dev Peprlexity: 8143.619140625
Epoch 3 / 10
100/100 [==============================] - 1s 12ms/step - loss: 4.5037 - accuracy: 0.1543 - perplexity: 4783.2236
Dev Loss: 4.503693103790283 Dev Acc: 0.15433269739151 Dev Peprlexity: 4783.2236328125
Epoch 4 / 10
100/10

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 10
hidden_size = 50
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 10
100/100 [==============================] - 2s 12ms/step - loss: 4.4638 - accuracy: 0.1455 - perplexity: 24139.1758
Dev Loss: 4.463803291320801 Dev Acc: 0.1454775482416153 Dev Peprlexity: 24139.17578125
Epoch 2 / 10
100/100 [==============================] - 1s 12ms/step - loss: 4.0982 - accuracy: 0.1866 - perplexity: 4176.0991
Dev Loss: 4.098181247711182 Dev Acc: 0.1865907609462738 Dev Peprlexity: 4176.09912109375
Epoch 3 / 10
100/100 [==============================] - 1s 8ms/step - loss: 3.9728 - accuracy: 0.2081 - perplexity: 2530.6182
Dev Loss: 3.9728448390960693 Dev Acc: 0.20809614658355713 Dev Peprlexity: 2530.6181640625
Epoch 4 / 10
10

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 10
hidden_size = 100
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 10
100/100 [==============================] - 2s 10ms/step - loss: 4.3084 - accuracy: 0.1841 - perplexity: 30388.1484
Dev Loss: 4.308379650115967 Dev Acc: 0.1840607225894928 Dev Peprlexity: 30388.1484375
Epoch 2 / 10
100/100 [==============================] - 1s 12ms/step - loss: 3.8305 - accuracy: 0.2302 - perplexity: 27271.1270
Dev Loss: 3.830456018447876 Dev Acc: 0.2302340269088745 Dev Peprlexity: 27271.126953125
Epoch 3 / 10
100/100 [==============================] - 1s 7ms/step - loss: 3.7491 - accuracy: 0.2505 - perplexity: 42611.1211
Dev Loss: 3.7490835189819336 Dev Acc: 0.25047439336776733 Dev Peprlexity: 42611.12109375
Epoch 4 / 10
100

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 10
hidden_size = 200
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 10
100/100 [==============================] - 1s 7ms/step - loss: 7.2942 - accuracy: 0.1025 - perplexity: 2940504.7500
Dev Loss: 7.294198989868164 Dev Acc: 0.10246679186820984 Dev Peprlexity: 2940504.75
Epoch 2 / 10
100/100 [==============================] - 1s 8ms/step - loss: 6.6733 - accuracy: 0.1581 - perplexity: 2883330.7500
Dev Loss: 6.673266410827637 Dev Acc: 0.1581277698278427 Dev Peprlexity: 2883330.75
Epoch 3 / 10
100/100 [==============================] - 1s 8ms/step - loss: 6.5695 - accuracy: 0.1777 - perplexity: 2866486.0000
Dev Loss: 6.569479942321777 Dev Acc: 0.1777356117963791 Dev Peprlexity: 2866486.0
Epoch 4 / 10
100/100 [====

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 10
hidden_size = 10
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 10
100/100 [==============================] - 1s 9ms/step - loss: 4.8221 - accuracy: 0.0734 - perplexity: 83939.5938
Dev Loss: 4.822131156921387 Dev Acc: 0.0733712837100029 Dev Peprlexity: 83939.59375
Epoch 2 / 10
100/100 [==============================] - 1s 8ms/step - loss: 4.5212 - accuracy: 0.1467 - perplexity: 18941.6973
Dev Loss: 4.521227836608887 Dev Acc: 0.1467425674200058 Dev Peprlexity: 18941.697265625
Epoch 3 / 10
100/100 [==============================] - 1s 8ms/step - loss: 4.2787 - accuracy: 0.1784 - perplexity: 2150.3076
Dev Loss: 4.278672218322754 Dev Acc: 0.17836812138557434 Dev Peprlexity: 2150.3076171875
Epoch 4 / 10
100/100 

In [ ]:
use_bert = False # to use the BERT embeddings
learning_rate = 0.1
epochs = 15
hidden_size = 100
embedding_size = 100
vocab, train_data = get_vocabulary_and_data(train_file, use_bert=use_bert)
_, dev_data = get_vocabulary_and_data(dev_file, use_bert=use_bert)

describe_data(train_data, batch_generator_lm(train_data, vocab, batch_size))

# from keras.initializers import Constant


with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

    for i in range(10):
        text = generate_text(language_model, vocab)
        print(text)

Data example: ['<s>', 'Next', ',', 'the', 'lamplighters', 'of', 'China', 'and', 'Siberia', 'would', 'enter', 'for', 'their', 'steps', 'in', 'the', 'dance', ',', 'and', 'then', 'they', 'too', 'would', 'be', 'waved', 'back', 'into', 'the', 'wings', '.', '</s>']
Data size 1362
Batch input shape: (50, 32)
Batch output shape: (50, 32, 2158)
Epoch 1 / 15
100/100 [==============================] - 2s 10ms/step - loss: 4.9786 - accuracy: 0.1746 - perplexity: 602839.8125
Dev Loss: 4.978583812713623 Dev Acc: 0.17457304894924164 Dev Peprlexity: 602839.8125
Epoch 2 / 15
100/100 [==============================] - 1s 8ms/step - loss: 4.4947 - accuracy: 0.2170 - perplexity: 472138.1562
Dev Loss: 4.494687557220459 Dev Acc: 0.21695129573345184 Dev Peprlexity: 472138.15625
Epoch 3 / 15
100/100 [==============================] - 1s 12ms/step - loss: 4.2962 - accuracy: 0.2188 - perplexity: 338101.5000
Dev Loss: 4.296202182769775 Dev Acc: 0.2188488245010376 Dev Peprlexity: 338101.5
Epoch 4 / 15
100/100 [==

In [ ]:
epochs = 7
with tf.device(device_name):
    # Implement your model here! ----------------------------------------------------------------------
    # Use the variables batch_size, hidden_size, embedding_size, dropout, epochs
    
    if use_bert:
        embedding_layer = BERT_Wrapper()
    else:
        embedding_matrix = load_pretrained_embeddings(glove_file, vocab)
        embedding_layer = Embedding(
            len(vocab),
            embedding_size,
            embeddings_initializer=Constant(embedding_matrix),
            trainable=False,
        )
    language_model = tf.keras.Sequential()
    input_size = len(vocab)
    output_size = len(vocab)

    drop_out_e = 0.25
    drop_out_lstm = 0.25
    drop_out_d = 0.25
    language_model.add(embedding_layer)
    language_model.add(tf.keras.layers.Dropout(drop_out_e))
    language_model.add(LSTM(hidden_size, return_sequences=True))# dropout

    language_model.add(TimeDistributed(Dense(output_size, activation='softmax')))
    language_model.add(tf.keras.layers.Dropout(drop_out_d))

    # ------------------------------------------------------------------------------------------------
    optimizer = tf.optimizers.Adam(learning_rate = learning_rate)
    language_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',perplexity])

    for i in range(epochs):
        print('Epoch',i+1,'/',epochs)
        # Training
        language_model.fit(batch_generator_lm(train_data, vocab, batch_size),
                                      epochs=1, steps_per_epoch=len(train_data)/batch_size)
        # Evaluation
        loss, acc, perp = language_model.evaluate(batch_generator_lm(dev_data, vocab),
                                                  steps=len(dev_data))
        print('Dev Loss:', loss, 'Dev Acc:', acc, 'Dev Peprlexity:', perp)

Epoch 1 / 7
100/100 [==============================] - 1s 8ms/step - loss: 4.1659 - accuracy: 0.2106 - perplexity: 119609.9922
Dev Loss: 4.1659040451049805 Dev Acc: 0.21062618494033813 Dev Peprlexity: 119609.9921875
Epoch 2 / 7
100/100 [==============================] - 1s 8ms/step - loss: 3.9683 - accuracy: 0.2359 - perplexity: 157780.4219
Dev Loss: 3.968287706375122 Dev Acc: 0.23592662811279297 Dev Peprlexity: 157780.421875
Epoch 3 / 7
100/100 [==============================] - 1s 12ms/step - loss: 3.8649 - accuracy: 0.2416 - perplexity: 151367.0000
Dev Loss: 3.8649444580078125 Dev Acc: 0.24161922931671143 Dev Peprlexity: 151367.0
Epoch 4 / 7
100/100 [==============================] - 1s 10ms/step - loss: 3.9053 - accuracy: 0.2467 - perplexity: 157699.4219
Dev Loss: 3.9053258895874023 Dev Acc: 0.24667932093143463 Dev Peprlexity: 157699.421875
Epoch 5 / 7
100/100 [==============================] - 1s 8ms/step - loss: 3.8589 - accuracy: 0.2467 - perplexity: 156436.2031
Dev Loss: 3.8588

In [ ]:
_, test_data = get_vocabulary_and_data(test_file, use_bert=use_bert)
loss, acc, perp = language_model.evaluate(batch_generator_lm(test_data, vocab),
                                          steps=len(test_data))
print('Test Loss:', loss, 'Test Acc:', acc, 'Test Peprlexity:', perp)

100/100 [==============================] - 1s 13ms/step - loss: 3.8480 - accuracy: 0.2702 - perplexity: 165402.7188
Test Loss: 3.8479981422424316 Test Acc: 0.2702205777168274 Test Peprlexity: 165402.71875
